In [1]:
%config Completer.use_jedi = False

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np

In [3]:
pcadf = pd.read_csv('pcadf.csv')
pcadf.drop('Unnamed: 0', axis=1, inplace=True)
pcadf.head()

,diagadhd,pc1,pc2,pc3,pc4,pc5,pc6,pc7
0,0,-6.316805,-3.993805,2.608238,-1.798007,0.423410,-0.186947,-0.877869
1,0,-3.483505,3.740298,3.714569,-1.483998,-0.931064,-1.223045,-0.586351
2,0,-5.072733,-1.089990,-2.044430,-0.990292,4.428686,3.542252,0.361166
3,0,9.102421,-0.533294,0.310204,6.574085,1.079962,-0.325457,0.911585
4,1,0.069205,-1.513308,0.906914,0.406834,-0.200324,2.936415,-0.700523


In [ ]:
pcadf = pcadf.iloc[:,0:4]
pcadf.head()

In [ ]:
pcadf.shape

In [ ]:
X = pcadf.iloc[0:117,1:].values
y = pcadf.iloc[0:117,0].values
print(X.shape, y.shape)

In [ ]:
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# split data into X and y
#X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,2:], data['diagadhd'], test_size=0.3)
training_set, test_set = train_test_split(pcadf, test_size = 0.2, random_state = 4)
X_train = training_set.iloc[:,1:].values
y_train = training_set.iloc[:,0].values
X_test = test_set.iloc[:,1:].values
y_test = test_set.iloc[:,0].values

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
log = LogisticRegression(C=0.18420699693267145, penalty='l2' )
rnd = RandomForestClassifier(n_estimators=80,bootstrap=False,max_depth=10,max_features='auto',min_samples_leaf=2, min_samples_split=5)
svm = SVC(C=1, kernel='linear',random_state=1 )

In [ ]:
voting = VotingClassifier(
    estimators=[('logistic_regression', log),('random_forest', rnd), ('support_vector_machine', svm)],
    voting='hard')

In [ ]:
voting.fit(X_train, y_train)

In [ ]:
scores = cross_val_score(voting, X_train, y_train, scoring='accuracy', cv=10)
print('10-CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
scores = cross_val_score(voting, X_test, y_test, scoring='accuracy', cv=10)
print('10-CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
# train set accuracy
for clf in (log, rnd, svm, voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
#test set accuracy
for clf in (log, rnd, svm, voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))